In [1]:
import pandas as pd 
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [2]:
df = pd.read_csv("/Users/meetpatel/Desktop/project_6/rossmann-store-sales.csv")
df.head()

,Unnamed: 0,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,...,Year,Month,Day,Weak of year,Quarter,Isweekend,BeginningOfMonth,MidOfMonth,EndOfMonth,Predicted_Sales
0,0,1,5,2015-07-31,5263.0,555.0,1.0,1,0,1,...,2015,7,31,31,3,0,0,1,1,5549.96
1,1,2,5,2015-07-31,6064.0,625.0,1.0,1,0,1,...,2015,7,31,31,3,0,0,1,1,6308.33
2,2,3,5,2015-07-31,8314.0,821.0,1.0,1,0,1,...,2015,7,31,31,3,0,0,1,1,8452.30
3,3,4,5,2015-07-31,13995.0,1498.0,1.0,1,0,1,...,2015,7,31,31,3,0,0,1,1,12404.01
4,4,5,5,2015-07-31,4822.0,559.0,1.0,1,0,1,...,2015,7,31,31,3,0,0,1,1,5646.14


In [3]:
train, test = train_test_split(df, test_size=0.2, random_state=42, shuffle=False)

In [4]:
features = ['Store', 'DayOfWeek', 'Promo', 'SchoolHoliday', 'Year', 'Month', 'Day', 'Quarter']
target = 'Sales'

X_train, y_train = train[features], train[target]
X_test, y_test = test[features], test[target]

In [5]:
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

/var/folders/89/88_l7y257kd5mnk20gl6rm5m0000gn/T/ipykernel_2212/903763345.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.fillna(0, inplace=True)
/var/folders/89/88_l7y257kd5mnk20gl6rm5m0000gn/T/ipykernel_2212/903763345.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.fillna(0, inplace=True)


In [6]:
for col in ['Store', 'DayOfWeek', 'Promo', 'SchoolHoliday']:
    X_train[col] = X_train[col].astype('category')
    X_test[col] = X_test[col].astype('category')

/var/folders/89/88_l7y257kd5mnk20gl6rm5m0000gn/T/ipykernel_2212/742946955.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[col] = X_train[col].astype('category')
/var/folders/89/88_l7y257kd5mnk20gl6rm5m0000gn/T/ipykernel_2212/742946955.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[col] = X_test[col].astype('category')


In [7]:
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42, enable_categorical=True)


In [8]:
# Hyperparameter tuning using GridSearchCV
param_grid = {
    'n_estimators': [100, 500],
    'max_depth': [3, 6, 10],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1]
}

grid_search = GridSearchCV(xgb_model, param_grid, cv=3, scoring='r2', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 72 candidates, totalling 216 fits


/opt/anaconda3/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


GridSearchCV(cv=3,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=True, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, ma...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=42, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.8, 1],
                         'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 6, 10], 'n_estimators': [100, 500],
                         'subsample': [0.8, 1]},
             scoring='r2', verbose=1)

In [9]:
best_xgb = grid_search.best_estimator_

In [10]:
best_xgb.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
             device=None, early_stopping_rounds=None, enable_categorical=True,
             eval_metric=None, feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.2, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=500,
             n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [11]:
y_pred = best_xgb.predict(X_test)

In [12]:
# Evaluate model
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"R2 Score: {r2:.4f}")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")

R2 Score: 0.8830
MAE: 776.92
RMSE: 1212.06


In [13]:
df.columns

Index(['Unnamed: 0', 'Store', 'DayOfWeek', 'Date', 'Sales', 'Customers',
       'Open', 'Promo', 'StateHoliday', 'SchoolHoliday', 'Id', 'Year', 'Month',
       'Day', 'Weak of year', 'Quarter', 'Isweekend', 'BeginningOfMonth',
       'MidOfMonth', 'EndOfMonth', 'Predicted_Sales'],
      dtype='object')

In [14]:
# Predict sales for the next six months
future_dates = pd.date_range(start=test['Date'].max(), periods=180, freq='D')
future_df = pd.DataFrame({
    'Year': future_dates.year,
    'Month': future_dates.month,
    'Day': future_dates.day,
    'DayOfWeek': future_dates.dayofweek,
    'Quarter': future_dates.quarter,
    'Store': test['Store'].unique()[0],  # Adjust if multiple stores need predictions
    'Promo': 0,  # Assuming no promo by default
    'SchoolHoliday': 0  # Adjust based on actual calendar
})

future_df['Store'] = future_df['Store'].astype('category')
future_df['DayOfWeek'] = future_df['DayOfWeek'].astype('category')

future_df = future_df[X_train.columns]

future_predictions = best_xgb.predict(future_df)

future_df['Predicted Sales'] = future_predictions


In [18]:
future_df.to_csv('/Users/meetpatel/Desktop/project_6/future prediction.csv')